# NFL Passrush/Blocker Section 3 (of 4)
# Metric Aggregation

This workbook aggregates the metrics, found on a game-play-player basis, to the overall player, team-play, team and game basis.  Includes both pass rusher (defensive) and vs pass rusher (pass blocker/offensive) aggregations.

#### Libraries Used

In [1]:
import nfl_frame_builder as nfl
import nfl_acquire_and_prep as acquire
import nfl_build_metrics as metrics

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

#### Generate random play analysis frames for testing

In [2]:
game, play, nflId, player_type = metrics.get_random_play_and_player()

In [3]:
week_num = nfl.get_week_of_game(game)
week_df = acquire.week(week_num)
scout_pass_block = acquire.scout_pass_block()
players_df = acquire.players()

In [4]:
qb_hold_time, point_of_scrimmage, analysis_frames = nfl.build_play_frames(game, play, week_df, nflId, scout_pass_block, player_type, v_type = 'PvP')

In [5]:
analysis_frames = metrics.build_metrics(analysis_frames, point_of_scrimmage, players_df)

In [6]:
analysis_frames.head()

,ball_x,ball_y,ball_next_x,ball_next_y,pass_rusher,pass_rusher_x,pass_rusher_y,pass_rusher_next_x,pass_rusher_next_y,pass_rusher_a,pass_blocker,pass_blocker_x,pass_blocker_y,pass_blocker_next_x,pass_blocker_next_y,pass_blocker_a,ball_distance_moved_x,ball_distance_moved_y,pass_rusher_distance_moved_x,pass_rusher_distance_moved_y,pass_blocker_distance_moved_x,pass_blocker_distance_moved_y,pass_rusher_to_ball_vector_x,pass_rusher_to_ball_vector_y,ball_to_pass_rusher_vector_x,ball_to_pass_rusher_vector_y,change_in_pass_rusher_to_ball_dist,pass_rusher_to_ball_dist_ratio,pass_rusher_pass_blocker_dist,colinearity,pursuit_factor,escape_factor,pass_rusher_force_to_ball,pursuit_vs_escape,pursuit1,pursuit2,pursuit3,pursuit4
0,1.17,-0.08,1.90,-0.09,41915,-0.83,4.93,-0.80,4.92,1.37,47814,1.28,2.85,1.30,2.85,0.67,0.73,-0.01,0.03,-0.01,0.02,0.00,0.0110,-0.0203,0.1685,-0.3126,0.2968,0.9479,2.9629,0.9529,0.7310,0.4864,103.84,-0.3320,1.1186,-0.3147,0.031625,-0.454752
1,1.90,-0.09,2.59,-0.10,41915,-0.80,4.92,-0.73,4.90,3.28,47814,1.30,2.85,1.36,2.87,1.42,0.69,-0.01,0.07,-0.02,0.06,0.02,0.0312,-0.0462,0.2157,-0.3249,0.3106,0.9482,2.9487,0.9654,0.7658,0.5652,260.45,-0.3342,1.0760,-0.3169,0.080786,-0.484297
2,2.59,-0.10,3.22,-0.12,41915,-0.73,4.90,-0.60,4.87,3.87,47814,1.36,2.87,1.46,2.89,2.13,0.63,-0.02,0.13,-0.03,0.10,0.02,0.0643,-0.0817,0.2424,-0.3167,0.2824,0.9551,2.9136,0.9810,0.7793,0.6328,312.72,-0.2949,1.0443,-0.2817,0.164946,-0.467860
3,3.22,-0.12,3.79,-0.14,41915,-0.60,4.87,-0.40,4.83,4.50,47814,1.46,2.89,1.61,2.93,2.25,0.57,-0.02,0.20,-0.04,0.15,0.04,0.1067,-0.1218,0.2467,-0.2926,0.2162,0.9667,2.8573,0.9869,0.7937,0.6710,370.35,-0.2208,1.0213,-0.2134,0.283907,-0.387130
4,3.79,-0.14,4.29,-0.16,41915,-0.40,4.83,-0.16,4.78,4.66,47814,1.61,2.93,1.79,3.00,2.06,0.50,-0.02,0.24,-0.05,0.18,0.07,0.1375,-0.1463,0.2339,-0.2597,0.1482,0.9777,2.7659,0.9863,0.8191,0.6985,395.79,-0.1487,1.0034,-0.1454,0.401226,-0.297162


----

## 1. Pull the metrics from a given play and create object

#### Main Function

In [7]:
def pull_metrics(analysis_frames, qb_hold_time):
    '''
    Consolidates metrics for a given play.
    
    Parameters:
        'analysis_frames' - Dataframe - Complete frames of the play.
        'qb_hold_time' - Float - How long the qb holds the ball (in seconds)
    Returns:
        'play_metrics' - Dictionary - Contains aggregated play metrics (averages over frames)
    '''
    # Pass Rusher Stats from analysis_frame
    pass_rusher = analysis_frames.pass_rusher.max()
    pass_rusher_average_a = round(analysis_frames.pass_rusher_a.mean(),4)
    colinearity = round(analysis_frames.colinearity.mean(),4)
    pursuit_factor = round(analysis_frames.pursuit_factor.mean(),4)
    force_to_ball = round(analysis_frames.pass_rusher_force_to_ball.mean(),4)
    #! -- Add any additional frame by frame metrics --
    pursuit_vs_escape = round(analysis_frames.pursuit_vs_escape.mean(),4)
    pursuit1 = round(analysis_frames.pursuit1.mean(),4)
    pursuit2 = round(analysis_frames.pursuit2.mean(),4)
    pursuit3_mean = round(analysis_frames.pursuit3.mean(),4)
    pursuit3_sum = round(analysis_frames.pursuit2.sum(),4)
    pursuit4 = round(analysis_frames.pursuit4.mean(),4)

    
    # List pass blockers in play - not needed in PvB
    pass_blockers = metrics.get_pass_blockers(analysis_frames)
    pass_blocker_nflId_list = []
    for pass_blocker in pass_blockers:
        pass_blocker_nflId_list.append(analysis_frames[pass_blocker].max())
        
    # Create dictionary object to hold results
    play_metrics = {'pass_rusher':pass_rusher,
                    'pass_rusher_average_a':pass_rusher_average_a,
                    'colinearity':colinearity,
                    'pursuit_factor':pursuit_factor,
                    'force_to_ball':force_to_ball,
                    'pursuit_vs_escape':pursuit_vs_escape,
                    'pursuit1':pursuit1,
                    'pursuit2':pursuit2,
                    'pursuit3_mean':pursuit3_mean,
                    'pursuit3_sum':pursuit3_sum,
                    'pursuit4':pursuit4,
                    'qb_hold_time':qb_hold_time,
                    'blocker_count':len(pass_blocker_nflId_list), # Not needed in PvB
                    'pass_blockers':pass_blocker_nflId_list} # Not needed in PvB
                    #! -- Add any additional frame by frame metrics --
        
    return play_metrics

i. Will be necessary to turn into dataframe for the future.

In [8]:
play_metrics = pd.DataFrame([pull_metrics(analysis_frames, qb_hold_time)])

In [9]:
play_metrics

,pass_rusher,pass_rusher_average_a,colinearity,pursuit_factor,force_to_ball,pursuit_vs_escape,pursuit1,pursuit2,pursuit3_mean,pursuit3_sum,pursuit4,qb_hold_time,blocker_count,pass_blockers
0,41915,2.7744,0.4466,0.7794,224.98,0.0358,-4.6974,0.0429,1.3872,1.4585,0.8876,3.5,1,[47814]


*Possible additional metric to explore later - comparing different levels of aggregation.*

In [10]:
# mass = players_df[players_df.nflId == analysis_frames.pass_rusher.max()].weight.iat[0] * 0.453592
# force_diff = play_metrics['force_to_ball'] - (play_metrics['pursuit_factor'] * mass * play_metrics['pass_rusher_average_a'])

## 2. Combine pulled metrics with each player-play outcome from the PFF scouting data.

##### Note: Went back and changed all previous funcitons that acquired data from dataframes within the functions to now pull that data in at a higher level, so that running aggregation functions didn't have to repeat these calls making the time to run them untenable.

i. Testing for a single week to further reduce compute resources needed - getting only the game-play combos from that week.

In [11]:
scout_pass_rush = acquire.scout_pass_rush()

In [12]:
week_game_list = week_df.game.unique()

In [13]:
scout_pass_rush = scout_pass_rush[scout_pass_rush.game.isin(week_game_list)]

In [14]:
def play_player_metrics_builder(scout_pass_rush, scout_pass_block, v_type, players_df, week_df):
    '''
    Given a specific week and its associated dataframe, create the metrics for each player in each play and then merge
    with player-play results from PFF's scouting reports.
    
    Note: For final analysis only need to do this for player_type = 'pass_rusher' as we can then pull the 'pass_blocker'
    metrics from these.
    
    Parameters:
        'scout_pass_rush' - Dataframe - Pass rusher PFF scouting data
        'scout_pass_block' - Dataframe - Pass blocker PFF scouting data
        'v_type' - String - The type of analysis to perform, defaulting to 'PvP'
        'players_df' - Dataframe - Contains player information, including weight
        'week_df' - Dataframe - Weekly frame by frame data for each play 
    Returns:
        'pass_rush_results' - Dataframe - Metrics for each player in each play, with pressure statistics from PFF scouting
    '''
    results = pd.DataFrame()
    
    week_game_list = week_df.game.unique()
    
    scout_pass_rush = scout_pass_rush[scout_pass_rush.game.isin(week_game_list)]
    
    for entry in scout_pass_rush.index:
        # Added a try except since there are errors when the snap events are missing
        try:
            game = scout_pass_rush.game.loc[entry]
            play = scout_pass_rush.play.loc[entry]
            nflId = scout_pass_rush.nflId.loc[entry]

            qb_hold_time, point_of_scrimmage, analysis_frames = nfl.build_play_frames(game,
                                                                                      play,
                                                                                      week_df,
                                                                                      nflId,
                                                                                      scout_pass_block,
                                                                                      player_type = 'pass_rusher',
                                                                                      v_type = v_type)

            analysis_frames = metrics.build_metrics(analysis_frames,
                                                    point_of_scrimmage,
                                                    players_df)

            play_metrics = pd.DataFrame([pull_metrics(analysis_frames, qb_hold_time)])
            
            play_metrics = pd.merge(scout_pass_rush[scout_pass_rush.index == entry],
                                    play_metrics,
                                    how = 'inner',
                                    left_on = 'nflId',
                                    right_on = 'pass_rusher')

            results = pd.concat([results, play_metrics])

            print(f'Added game|play|nflId = {game}|{play}|{nflId}')
            
        except:
            print(f'*****Frame event error for game|play|nflId = {game}|{play}|{nflId}')
    
    pass_rush_results = results.drop(columns = ['pass_rusher'])
    
    return pass_rush_results

ii. Running the function

In [15]:
# Load the parameters
week_df = acquire.week(1)
players_df = acquire.players()
scout_pass_rush = acquire.scout_pass_rush()
scout_pass_block = acquire.scout_pass_block()

In [16]:
# Commented out to prevent running
# pass_rush_results = play_player_metrics_builder(scout_pass_rush, scout_pass_block, 'PvP', players_df, week_df)
# pass_rush_results

# FINAL FUNCTION!

In [17]:
def all_week_pass_rush_results(start_week = 1, end_week = 8):
    '''
    Self-contained function that pulls in all the metrics for desired weeks and outputs pass_rush_results.
    
    Note: Week range for Data Bowl is 1-8.  Can use 1-6 for training data for any ML analysis performed.
    
    Parameters:
        'start_week' - Integer - Week to start building from (inclusive)
        'end_week' - Integer - Week to build to (inclusive)
    Returns:
        'all_results' - Dataframe - Metrics for each player-play for given weeks
        ***.csv of results saved to folder***
    '''
    # Load the data used within the sub-functions
    players_df = acquire.players()
    scout_pass_rush = acquire.scout_pass_rush()
    scout_pass_block = acquire.scout_pass_block()
    # Default to 'PvP'
    v_type = 'PvP'
    
    # Create empty dataframe to hold results
    all_results = pd.DataFrame()
    
    # Run through weeks to build for    
    for i in range(start_week, end_week + 1):
        print('2021 NFL Week:',i)
        # Acquire that week's frame data
        week_df = acquire.week(i)
    
        pass_rush_results = play_player_metrics_builder(scout_pass_rush, scout_pass_block, v_type, players_df, week_df)
        
        all_results = pd.concat([all_results, pass_rush_results])
        
    # Save to a csv for easier later use
    filename = f'metric_results_weeks_{start_week}_through_{end_week}.csv'
    all_results.to_csv(filename, index = False)
        
    return all_results        

In [18]:
# Commented out to prevent accidental running - takes a few minutes
all_results = all_week_pass_rush_results(1,1)

2021 NFL Week: 1
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|97|41263
Added game|play|nflId = 2021090900|97|42403
Added game|play|nflId = 2021090900|97|44955
Added game|play|nflId = 2021090900|97|53441
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|97|53504
Added game|play|nflId = 2021090900|137|35441
Added game|play|nflId = 2021090900|137|35454
Added game|play|nflId = 2021090900|137|40074
Added game|play|nflId = 2021090900|137|46081
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|137|53461
Added game|play|nflId = 2021090900|187|35441
Added game|play|nflId = 2021090900|187|35454
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|187|38588
Added game|play|nflId = 2021090900|187|46081
Added game|play|nflId = 2021090900|282|34777
Added

Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|1918|41915
Added game|play|nflId = 2021090900|1918|42560
Added game|play|nflId = 2021090900|1918|47882
Added game|play|nflId = 2021090900|1940|35441
Added game|play|nflId = 2021090900|1940|35454
Added game|play|nflId = 2021090900|1940|41915
Added game|play|nflId = 2021090900|1940|46081
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|2008|41263
Added game|play|nflId = 2021090900|2008|41363
Added game|play|nflId = 2021090900|2008|44955
Added game|play|nflId = 2021090900|2008|46185
Added game|play|nflId = 2021090900|2032|41263
Added game|play|nflId = 2021090900|2032|41363
Added game|play|nflId = 2021090900|2032|44955
Added game|play|nflId = 2021090900|2032|46185
Added game|play|nflId = 2021090900|2088|35441
Added game|play|nflId = 2021090900|2088|35454
Added game|play|nflId = 2021090900|2088|46081
Added game|play|nf

Added game|play|nflId = 2021090900|3462|42403
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|3462|43323
Added game|play|nflId = 2021090900|3462|53504
Added game|play|nflId = 2021090900|3588|41363
Added game|play|nflId = 2021090900|3588|53504
Added game|play|nflId = 2021090900|3633|44892
Added game|play|nflId = 2021090900|3633|44955
Added game|play|nflId = 2021090900|3633|46185
Added game|play|nflId = 2021090900|3633|53621
Added game|play|nflId = 2021090900|3655|41263
Added game|play|nflId = 2021090900|3655|42403
Added game|play|nflId = 2021090900|3655|46185
Added game|play|nflId = 2021090900|3655|53441
Added game|play|nflId = 2021090900|3655|53504
Added game|play|nflId = 2021090900|3724|34777
Added game|play|nflId = 2021090900|3724|35454
Added game|play|nflId = 2021090900|3724|40074
Added game|play|nflId = 2021090900|3724|41915
Added game|play|nflId = 2021090900|3724|42560
Added game|play|nflId = 2021090900|3746|35441
Add

Added game|play|nflId = 2021091200|864|43405
Added game|play|nflId = 2021091200|864|44826
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091200|864|46219
Added game|play|nflId = 2021091200|864|53502
Added game|play|nflId = 2021091200|966|37089
Added game|play|nflId = 2021091200|966|38542
Added game|play|nflId = 2021091200|966|43378
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091200|966|53663
Added game|play|nflId = 2021091200|1023|35452
Added game|play|nflId = 2021091200|1023|38542
Added game|play|nflId = 2021091200|1023|43378
Added game|play|nflId = 2021091200|1023|46199
Added game|play|nflId = 2021091200|1085|35452
Added game|play|nflId = 2021091200|1085|38542
Added game|play|nflId = 2021091200|1085|43378
Added game|play|nflId = 2021091200|1085|46199
Added game|play|nflId = 2021091200|1153|40095
Added game|play|nflId = 2021091200|1153|42346
Added game|play|nflId = 20

Added game|play|nflId = 2021091200|3599|52542
Added game|play|nflId = 2021091200|3659|42480
Added game|play|nflId = 2021091200|3659|42497
Added game|play|nflId = 2021091200|3659|43361
Added game|play|nflId = 2021091200|3659|46269
Added game|play|nflId = 2021091200|3659|53611
Added game|play|nflId = 2021091200|3728|37089
Added game|play|nflId = 2021091200|3728|46199
Added game|play|nflId = 2021091200|3728|53502
Added game|play|nflId = 2021091200|3728|53618
Added game|play|nflId = 2021091200|3752|37089
Added game|play|nflId = 2021091200|3752|46199
Added game|play|nflId = 2021091200|3752|53502
Added game|play|nflId = 2021091200|3752|53618
Added game|play|nflId = 2021091200|3776|37089
Added game|play|nflId = 2021091200|3776|46199
Added game|play|nflId = 2021091200|3776|53502
Added game|play|nflId = 2021091200|3776|53618
Added game|play|nflId = 2021091200|3798|37089
Added game|play|nflId = 2021091200|3798|43405
Added game|play|nflId = 2021091200|3798|46199
Added game|play|nflId = 2021091200

...

Added game|play|nflId = 2021091212|2703|47800
Added game|play|nflId = 2021091212|2804|37075
Added game|play|nflId = 2021091212|2804|41464
Added game|play|nflId = 2021091212|2804|47854
Added game|play|nflId = 2021091212|2804|48089
Added game|play|nflId = 2021091212|2857|37075
Added game|play|nflId = 2021091212|2857|40347
Added game|play|nflId = 2021091212|2857|47854
Added game|play|nflId = 2021091212|2857|48089
Added game|play|nflId = 2021091212|2883|37075
Added game|play|nflId = 2021091212|2883|41464
Added game|play|nflId = 2021091212|2883|47016
Added game|play|nflId = 2021091212|2883|47854
Added game|play|nflId = 2021091212|2883|48089
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091212|2905|37075
Added game|play|nflId = 2021091212|2905|41464
Added game|play|nflId = 2021091212|2905|47854
Added game|play|nflId = 2021091212|2905|48089
Added game|play|nflId = 2021091212|2947|40347
Added game|play|nflId = 2021091212|2947|41449
Add

Added game|play|nflId = 2021091213|583|46214
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091213|607|37087
Added game|play|nflId = 2021091213|607|38619
Added game|play|nflId = 2021091213|607|41231
Added game|play|nflId = 2021091213|607|46214
Added game|play|nflId = 2021091213|684|43335
Added game|play|nflId = 2021091213|684|47917
Added game|play|nflId = 2021091213|684|47939
Added game|play|nflId = 2021091213|684|52492
Added game|play|nflId = 2021091213|684|52846
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091213|775|41239
Added game|play|nflId = 2021091213|775|43298
Added game|play|nflId = 2021091213|775|46264
Added game|play|nflId = 2021091213|775|47939
Added game|play|nflId = 2021091213|804|41239
Added game|play|nflId = 2021091213|804|43298
Added game|play|nflId = 2021091213|804|46264
Added game|play|nflId = 2021091213|804|47939
Added game|play|nflId = 2021091213|8

Added game|play|nflId = 2021091213|2905|46191
Added game|play|nflId = 2021091213|2905|46264
Added game|play|nflId = 2021091213|3054|38619
Added game|play|nflId = 2021091213|3054|42443
Added game|play|nflId = 2021091213|3054|46077
Added game|play|nflId = 2021091213|3154|37087
Added game|play|nflId = 2021091213|3154|38619
Added game|play|nflId = 2021091213|3154|42443
Added game|play|nflId = 2021091213|3154|52563
Added game|play|nflId = 2021091213|3154|53679
Added game|play|nflId = 2021091213|3224|38619
Added game|play|nflId = 2021091213|3224|41231
Added game|play|nflId = 2021091213|3224|41279
Added game|play|nflId = 2021091213|3224|42443
Added game|play|nflId = 2021091213|3284|37087
Added game|play|nflId = 2021091213|3284|38619
Added game|play|nflId = 2021091213|3284|41231
Added game|play|nflId = 2021091213|3284|46214
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091213|3351|41239
Added game|play|nflId = 2021091213|3351|43298
Add

Added game|play|nflId = 2021091300|1485|41336
Added game|play|nflId = 2021091300|1485|52578
Added game|play|nflId = 2021091300|1485|53460
Added game|play|nflId = 2021091300|1507|33131
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091300|1507|37145
Added game|play|nflId = 2021091300|1507|37240
Added game|play|nflId = 2021091300|1507|41336
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091300|1553|33131
Added game|play|nflId = 2021091300|1553|37145
Added game|play|nflId = 2021091300|1553|37240
Added game|play|nflId = 2021091300|1553|40042
Added game|play|nflId = 2021091300|1553|44999
Added game|play|nflId = 2021091300|1553|46259
Added game|play|nflId = 2021091300|1553|46958
Added game|play|nflId = 2021091300|1655|39997
Added game|play|nflId = 2021091300|1655|42535
Added game|play|nflId = 2021091300|1655|43354
Added game|play|nflId = 2021091300|1655|47889
Added game|play|nf

Added game|play|nflId = 2021091300|3385|52479
Added game|play|nflId = 2021091300|3385|52578
Added game|play|nflId = 2021091300|3385|53460
Added game|play|nflId = 2021091300|3478|39997
Added game|play|nflId = 2021091300|3478|42535
Added game|play|nflId = 2021091300|3478|43354
Added game|play|nflId = 2021091300|3478|47889
Added game|play|nflId = 2021091300|3544|35442
Added game|play|nflId = 2021091300|3544|43436
Added game|play|nflId = 2021091300|3544|44815
Added game|play|nflId = 2021091300|3544|47889
Added game|play|nflId = 2021091300|3619|33131
Added game|play|nflId = 2021091300|3619|37145
Added game|play|nflId = 2021091300|3619|40042
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091300|3619|44828
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091300|3619|52506
Added game|play|nflId = 2021091300|3664|33131
Added game|play|nflId = 2021091300|3664|37145
Added game|play|nf

In [19]:
results_check = pd.read_csv('metric_results_weeks_1_through_1.csv')
results_check

,game,play,nflId,position,hit,hurry,sack,pressure,pass_rusher_average_a,colinearity,pursuit_factor,force_to_ball,pursuit_vs_escape,pursuit1,pursuit2,pursuit3_mean,pursuit3_sum,pursuit4,qb_hold_time,blocker_count,pass_blockers
0,2021090900,97,41263,LEO,0,1,0,1,1.6845,0.5703,0.7769,139.2433,-0.0259,-4.9706,0.0566,2.8228,1.8690,2.0332,3.4,0,[]
1,2021090900,97,42403,ROLB,0,1,0,1,2.5479,0.8340,0.8895,238.3558,0.0799,-7.1719,0.1449,2.9268,4.7824,2.2977,3.4,1,[42377]
2,2021090900,97,44955,DRT,0,0,0,0,1.3261,0.3828,0.1758,18.2900,-0.1393,-0.7108,-0.1279,1.1878,-4.2197,0.2978,3.4,2,"[40151, 42404]"
3,2021090900,97,53441,LILB,0,1,0,1,2.8409,0.7342,0.9013,251.3415,-0.0701,-4.8279,0.0278,2.6664,0.9184,1.8429,3.4,2,"[46163, 52421]"
4,2021090900,97,53504,RE,0,1,0,1,1.5467,0.7741,0.7099,121.5688,-0.0599,-4.2989,-0.0453,2.8965,-1.4943,2.1886,3.4,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983,2021091300,4845,37240,ROLB,0,0,0,0,2.8082,0.7944,0.9335,293.0223,-0.1195,0.2990,-0.1168,0.6206,-2.5692,-0.2485,2.3,1,[46084]
4984,2021091300,4845,40042,NLT,0,0,0,0,1.1027,0.9967,0.9843,150.7200,-0.2184,0.9982,-0.2036,0.5453,-4.4789,-0.4389,2.3,1,[44972]
4985,2021091300,4845,46259,LOLB,0,0,0,0,2.8627,0.8616,0.9264,230.2873,0.1953,-5.9391,0.2116,1.0865,4.6543,0.5010,2.3,1,[43362]
4986,2021091300,4845,52506,RCB,0,0,0,0,2.3314,0.8805,0.9291,218.7155,0.1783,-5.5118,0.1864,1.0960,4.0998,0.4535,2.3,0,[]


### Test .py functions

In [20]:
import nfl_use_metrics as use

In [21]:
results = use.all_week_pass_rush_results(1,8)

2021 NFL Week: 1
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|97|41263
Added game|play|nflId = 2021090900|97|42403
Added game|play|nflId = 2021090900|97|44955
Added game|play|nflId = 2021090900|97|53441
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|97|53504
Added game|play|nflId = 2021090900|137|35441
Added game|play|nflId = 2021090900|137|35454
Added game|play|nflId = 2021090900|137|40074
Added game|play|nflId = 2021090900|137|46081
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|137|53461
Added game|play|nflId = 2021090900|187|35441
Added game|play|nflId = 2021090900|187|35454
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|187|38588
Added game|play|nflId = 2021090900|187|46081
Added game|play|nflId = 2021090900|282|34777
Added

Added game|play|nflId = 2021090900|1918|47882
Added game|play|nflId = 2021090900|1940|35441
Added game|play|nflId = 2021090900|1940|35454
Added game|play|nflId = 2021090900|1940|41915
Added game|play|nflId = 2021090900|1940|46081
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021090900|2008|41263
Added game|play|nflId = 2021090900|2008|41363
Added game|play|nflId = 2021090900|2008|44955
Added game|play|nflId = 2021090900|2008|46185
Added game|play|nflId = 2021090900|2032|41263
Added game|play|nflId = 2021090900|2032|41363
Added game|play|nflId = 2021090900|2032|44955
Added game|play|nflId = 2021090900|2032|46185
Added game|play|nflId = 2021090900|2088|35441
Added game|play|nflId = 2021090900|2088|35454
Added game|play|nflId = 2021090900|2088|46081
Added game|play|nflId = 2021090900|2088|47890
Added game|play|nflId = 2021090900|2129|35441
Added game|play|nflId = 2021090900|2129|35454
Added game|play|nflId = 2021090900|2129|46081
Add

Added game|play|nflId = 2021090900|3588|53504
Added game|play|nflId = 2021090900|3633|44892
Added game|play|nflId = 2021090900|3633|44955
Added game|play|nflId = 2021090900|3633|46185
Added game|play|nflId = 2021090900|3633|53621
Added game|play|nflId = 2021090900|3655|41263
Added game|play|nflId = 2021090900|3655|42403
Added game|play|nflId = 2021090900|3655|46185
Added game|play|nflId = 2021090900|3655|53441
Added game|play|nflId = 2021090900|3655|53504
Added game|play|nflId = 2021090900|3724|34777
Added game|play|nflId = 2021090900|3724|35454
Added game|play|nflId = 2021090900|3724|40074
Added game|play|nflId = 2021090900|3724|41915
Added game|play|nflId = 2021090900|3724|42560
Added game|play|nflId = 2021090900|3746|35441
Added game|play|nflId = 2021090900|3746|35454
Added game|play|nflId = 2021090900|3746|41915
Added game|play|nflId = 2021090900|3746|46081
Added game|play|nflId = 2021090900|3793|44892
Added game|play|nflId = 2021090900|3793|44955
Added game|play|nflId = 2021090900

Added game|play|nflId = 2021091200|864|53502
Added game|play|nflId = 2021091200|966|37089
Added game|play|nflId = 2021091200|966|38542
Added game|play|nflId = 2021091200|966|43378
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021091200|966|53663
Added game|play|nflId = 2021091200|1023|35452
Added game|play|nflId = 2021091200|1023|38542
Added game|play|nflId = 2021091200|1023|43378
Added game|play|nflId = 2021091200|1023|46199
Added game|play|nflId = 2021091200|1085|35452
Added game|play|nflId = 2021091200|1085|38542
Added game|play|nflId = 2021091200|1085|43378
Added game|play|nflId = 2021091200|1085|46199
Added game|play|nflId = 2021091200|1153|40095
Added game|play|nflId = 2021091200|1153|42346
Added game|play|nflId = 2021091200|1153|42480
Added game|play|nflId = 2021091200|1153|43361
Added game|play|nflId = 2021091200|1182|40095
Added game|play|nflId = 2021091200|1182|42480
Added game|play|nflId = 2021091200|1182|43361
Added ga

Added game|play|nflId = 2021091200|3599|52542
Added game|play|nflId = 2021091200|3659|42480
Added game|play|nflId = 2021091200|3659|42497
Added game|play|nflId = 2021091200|3659|43361
Added game|play|nflId = 2021091200|3659|46269
Added game|play|nflId = 2021091200|3659|53611
Added game|play|nflId = 2021091200|3728|37089
Added game|play|nflId = 2021091200|3728|46199
Added game|play|nflId = 2021091200|3728|53502
Added game|play|nflId = 2021091200|3728|53618
Added game|play|nflId = 2021091200|3752|37089
Added game|play|nflId = 2021091200|3752|46199
Added game|play|nflId = 2021091200|3752|53502
Added game|play|nflId = 2021091200|3752|53618
Added game|play|nflId = 2021091200|3776|37089
Added game|play|nflId = 2021091200|3776|46199
Added game|play|nflId = 2021091200|3776|53502
Added game|play|nflId = 2021091200|3776|53618
Added game|play|nflId = 2021091200|3798|37089
Added game|play|nflId = 2021091200|3798|43405
Added game|play|nflId = 2021091200|3798|46199
Added game|play|nflId = 2021091200

...

Added game|play|nflId = 2021103111|2909|35441
Added game|play|nflId = 2021103111|2909|40074
Added game|play|nflId = 2021103111|2909|41915
Added game|play|nflId = 2021103111|2909|46081
Added game|play|nflId = 2021103111|3019|42553
Added game|play|nflId = 2021103111|3019|44871
Added game|play|nflId = 2021103111|3019|44905
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021103111|3098|34465
Added game|play|nflId = 2021103111|3098|37097
Added game|play|nflId = 2021103111|3098|43409
Added game|play|nflId = 2021103111|3098|46083
Added game|play|nflId = 2021103111|3098|47888
Added game|play|nflId = 2021103111|3123|37097
Added game|play|nflId = 2021103111|3123|43409
Added game|play|nflId = 2021103111|3123|46083
Added game|play|nflId = 2021103111|3198|35441
Added game|play|nflId = 2021103111|3198|35454
Added game|play|nflId = 2021103111|3198|41915
Added game|play|nflId = 2021103111|3198|46081
Added game|play|nflId = 2021103111|3246|35441
Add

Added game|play|nflId = 2021103112|685|53504
Added game|play|nflId = 2021103112|788|39959
Added game|play|nflId = 2021103112|788|41235
Added game|play|nflId = 2021103112|788|42431
Added game|play|nflId = 2021103112|788|47973
Added game|play|nflId = 2021103112|788|52525
Added game|play|nflId = 2021103112|821|39959
Added game|play|nflId = 2021103112|821|42431
Added game|play|nflId = 2021103112|821|47973
Added game|play|nflId = 2021103112|821|52525
Added game|play|nflId = 2021103112|939|44867
Added game|play|nflId = 2021103112|939|52525
Added game|play|nflId = 2021103112|939|52538
Added game|play|nflId = 2021103112|939|52633
Added game|play|nflId = 2021103112|1021|42403
Added game|play|nflId = 2021103112|1021|42667
Added game|play|nflId = 2021103112|1021|46185
Added game|play|nflId = 2021103112|1021|53504
Added game|play|nflId = 2021103112|1045|42403
Added game|play|nflId = 2021103112|1045|43533
Added game|play|nflId = 2021103112|1045|44892
Added game|play|nflId = 2021103112|1045|53504
Ad

Added game|play|nflId = 2021103112|2860|42388
Added game|play|nflId = 2021103112|2860|44867
Added game|play|nflId = 2021103112|2860|52525
Added game|play|nflId = 2021103112|2860|52538
Added game|play|nflId = 2021103112|2860|52633
Added game|play|nflId = 2021103112|2938|41235
Added game|play|nflId = 2021103112|2938|44867
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021103112|2938|52525
Added game|play|nflId = 2021103112|2938|52538
Added game|play|nflId = 2021103112|2938|52633
Added game|play|nflId = 2021103112|2968|35539
Added game|play|nflId = 2021103112|2968|39959
Added game|play|nflId = 2021103112|2968|42388
Added game|play|nflId = 2021103112|2968|52525
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021103112|2968|52633
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021103112|3024|42403
Added game|play|nflId = 202110311

Added game|play|nflId = 2021110100|393|53479
Added game|play|nflId = 2021110100|393|53645
Added game|play|nflId = 2021110100|441|42406
Added game|play|nflId = 2021110100|441|43326
Added game|play|nflId = 2021110100|441|43338
Added game|play|nflId = 2021110100|441|46144
Added game|play|nflId = 2021110100|531|42349
Added game|play|nflId = 2021110100|531|53479
Added game|play|nflId = 2021110100|531|53645
Added game|play|nflId = 2021110100|587|42406
Added game|play|nflId = 2021110100|587|43326
Added game|play|nflId = 2021110100|587|43338
Added game|play|nflId = 2021110100|587|46144
Added game|play|nflId = 2021110100|724|42349
Added game|play|nflId = 2021110100|724|47800
Added game|play|nflId = 2021110100|724|47878
Added game|play|nflId = 2021110100|724|53645
Added game|play|nflId = 2021110100|753|42349
Added game|play|nflId = 2021110100|753|47800
Added game|play|nflId = 2021110100|753|47878
Added game|play|nflId = 2021110100|753|53645
Added game|play|nflId = 2021110100|775|42349
Added game

Added game|play|nflId = 2021110100|3016|52585
Added game|play|nflId = 2021110100|3016|52792
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021110100|3089|42406
Added game|play|nflId = 2021110100|3089|43326
Added game|play|nflId = 2021110100|3089|52585
Added game|play|nflId = 2021110100|3089|52792
Added game|play|nflId = 2021110100|3150|42406
Added game|play|nflId = 2021110100|3150|43326
Added game|play|nflId = 2021110100|3150|52471
Added game|play|nflId = 2021110100|3150|52585
Added game|play|nflId = 2021110100|3150|52792
Added game|play|nflId = 2021110100|3150|53487
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 2021110100|3195|40051
Added game|play|nflId = 2021110100|3195|43326
Added game|play|nflId = 2021110100|3195|43338
Added game|play|nflId = 2021110100|3195|52471
Player did not have any opponents listed, cannot do PvP, showing PvB instead
Added game|play|nflId = 202111010

In [22]:
results

,game,play,nflId,position,hit,hurry,sack,pressure,pass_rusher_average_a,colinearity,pursuit_factor,force_to_ball,pursuit_vs_escape,pursuit1,pursuit2,pursuit3_mean,pursuit3_sum,pursuit4,qb_hold_time,blocker_count,pass_blockers
0,2021090900,97,41263,LEO,0,1,0,1,1.6845,0.5703,0.7769,139.2433,-0.0259,-4.9706,0.0566,2.8228,1.8690,2.0332,3.4,0,[]
0,2021090900,97,42403,ROLB,0,1,0,1,2.5479,0.8340,0.8895,238.3558,0.0799,-7.1719,0.1449,2.9268,4.7824,2.2977,3.4,1,[42377]
0,2021090900,97,44955,DRT,0,0,0,0,1.3261,0.3828,0.1758,18.2900,-0.1393,-0.7108,-0.1279,1.1878,-4.2197,0.2978,3.4,2,"[40151, 42404]"
0,2021090900,97,53441,LILB,0,1,0,1,2.8409,0.7342,0.9013,251.3415,-0.0701,-4.8279,0.0278,2.6664,0.9184,1.8429,3.4,2,"[46163, 52421]"
0,2021090900,97,53504,RE,0,1,0,1,1.5467,0.7741,0.7099,121.5688,-0.0599,-4.2989,-0.0453,2.8965,-1.4943,2.1886,3.4,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2021110100,4411,52585,REO,0,0,0,0,1.9500,0.9689,0.7519,157.6532,0.0798,-5.1314,0.0877,1.4959,2.1923,0.8335,2.6,1,[52507]
0,2021110100,4433,42406,ROLB,0,0,1,1,2.3549,0.4351,0.8122,185.1724,0.1096,-5.0615,0.1298,1.3372,5.8413,0.6421,4.6,1,[44835]
0,2021110100,4433,43326,LE,0,0,0,0,1.7853,0.5652,0.2214,21.0267,-0.0900,-0.2789,-0.0867,0.3970,-3.9035,-0.3297,4.6,2,"[43425, 46103]"
0,2021110100,4433,43338,RE,0,1,0,1,1.7389,-0.0042,0.3107,61.9756,-0.0388,-2.8818,-0.0289,0.7019,-1.3015,-0.0485,4.6,2,"[43695, 52507]"


In [23]:
results_check = pd.read_csv('metric_results_weeks_1_through_8.csv')
results_check

,game,play,nflId,position,hit,hurry,sack,pressure,pass_rusher_average_a,colinearity,pursuit_factor,force_to_ball,pursuit_vs_escape,pursuit1,pursuit2,pursuit3_mean,pursuit3_sum,pursuit4,qb_hold_time,blocker_count,pass_blockers
0,2021090900,97,41263,LEO,0,1,0,1,1.6845,0.5703,0.7769,139.2433,-0.0259,-4.9706,0.0566,2.8228,1.8690,2.0332,3.4,0,[]
1,2021090900,97,42403,ROLB,0,1,0,1,2.5479,0.8340,0.8895,238.3558,0.0799,-7.1719,0.1449,2.9268,4.7824,2.2977,3.4,1,[42377]
2,2021090900,97,44955,DRT,0,0,0,0,1.3261,0.3828,0.1758,18.2900,-0.1393,-0.7108,-0.1279,1.1878,-4.2197,0.2978,3.4,2,"[40151, 42404]"
3,2021090900,97,53441,LILB,0,1,0,1,2.8409,0.7342,0.9013,251.3415,-0.0701,-4.8279,0.0278,2.6664,0.9184,1.8429,3.4,2,"[46163, 52421]"
4,2021090900,97,53504,RE,0,1,0,1,1.5467,0.7741,0.7099,121.5688,-0.0599,-4.2989,-0.0453,2.8965,-1.4943,2.1886,3.4,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36211,2021110100,4411,52585,REO,0,0,0,0,1.9500,0.9689,0.7519,157.6532,0.0798,-5.1314,0.0877,1.4959,2.1923,0.8335,2.6,1,[52507]
36212,2021110100,4433,42406,ROLB,0,0,1,1,2.3549,0.4351,0.8122,185.1724,0.1096,-5.0615,0.1298,1.3372,5.8413,0.6421,4.6,1,[44835]
36213,2021110100,4433,43326,LE,0,0,0,0,1.7853,0.5652,0.2214,21.0267,-0.0900,-0.2789,-0.0867,0.3970,-3.9035,-0.3297,4.6,2,"[43425, 46103]"
36214,2021110100,4433,43338,RE,0,1,0,1,1.7389,-0.0042,0.3107,61.9756,-0.0388,-2.8818,-0.0289,0.7019,-1.3015,-0.0485,4.6,2,"[43695, 52507]"
